In [2]:
# python3 -m greenferries.aishub.create_gpx

import sqlite3, os, sys, json
import pandas as pd
# current dir
DB_PATH = os.path.join(os.path.abspath(''), "..", "data_files", "aishub.db")


con = sqlite3.connect(DB_PATH)
con.row_factory = sqlite3.Row
cur = con.cursor()

ships = cur.execute("""
    SELECT ships.imo, ships.name
    FROM ships
    INNER JOIN trackpoints ON trackpoints.imo = ships.imo
""").fetchall()

In [4]:
import folium
m = folium.Map(location=[0, 0], zoom_start=2)

con.row_factory = None
cur = con.cursor()
coordinates_by_imo = {
    ship: cur.execute(
            """
                SELECT latitude, longitude
                FROM trackpoints
                WHERE imo = ?
                ORDER BY time
            """,
            (ship["imo"],),
        ).fetchall()
    for ship in ships
}
for ship, coordinates in coordinates_by_imo.items():
    folium.PolyLine(coordinates, tooltip=f"{ship['name']} {ship['imo']}").add_to(m)

# center of all coordinates
latitudes = [coordinate[0] for coordinates in coordinates_by_imo.values() for coordinate in coordinates]
longitudes = [coordinate[1] for coordinates in coordinates_by_imo.values() for coordinate in coordinates]
m.fit_bounds([[min(latitudes), min(longitudes)], [max(latitudes), max(longitudes)]])
m